Let's practice loading and exploring some data! 🤓

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://github.com/mbrudd/csci290/raw/main/data/stroke.csv")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["stroke"].unique()

In [ ]:
df['ever_married'].unique()

In [ ]:
df['Residence_type'].unique()

In [ ]:
df["stroke"].value_counts()

In [ ]:
df["stroke"].value_counts( normalize=True )

In [ ]:
df["stroke"].mean()

In [ ]:
df.plot(x="age", y="stroke", kind="scatter")

In [ ]:
df["age"].describe()

In [ ]:
df["age_cat"] = pd.cut( df["age"], bins = [0,25,45,61,100], labels = ["Young","Adult","Middle-aged","Old"] )

In [ ]:
df["age_cat"].value_counts()

In [ ]:
df.groupby("age_cat", observed=True)["stroke"].mean()

In [ ]:
df.groupby("heart_disease")["stroke"].mean()

In [ ]:
df.groupby(["heart_disease","hypertension", "ever_married"])["stroke"].mean()

# Build and assess a simple classifier

In [ ]:
df["prediction"] = df["age"] > 25

In [ ]:
# import sys
# !{sys.executable} -m pip install scikit-learn

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix( df["stroke"], df["prediction"])
cm

In [ ]:
ConfusionMatrixDisplay( cm, display_labels=["no stroke","stroke"]).plot()

In [ ]:
tn, fp, fn, tp = confusion_matrix( df["stroke"], df["prediction"]).ravel()
tn, fp, fn, tp

In [ ]:
import sklearn.metrics as metrics

In [ ]:
metrics.recall_score( df["stroke"], df["prediction"])

In [ ]:
tp / (tp + fn)

In [ ]:
metrics.precision_score( df["stroke"], df["prediction"])

In [ ]:
tp / (tp + fp)

In [ ]:
def precision_vs_recall( threshold ):
  df["prediction"] = df["age"] > threshold
  return metrics.precision_score( df["stroke"], df["prediction"]), metrics.recall_score( df["stroke"], df["prediction"])

In [ ]:
precision_vs_recall( 61 )

In [ ]:
precisions = []
recalls = []
thresholds = range(0,82)
for threshold in thresholds:
  precision, recall = precision_vs_recall( threshold )
  precisions.append( precision )
  recalls.append( recall )

In [ ]:
pr = pd.DataFrame( {
    "threshold": thresholds,
    "precision": precisions,
    "recall": recalls
})

In [ ]:
pr.tail()

In [ ]:
pr.plot( x="threshold", y=["precision","recall"]);

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2)

In [ ]:
train_set.columns

In [ ]:
train_set = train_set[["gender","age","hypertension","heart_disease","smoking_status","stroke"]]

In [ ]:
test_set = test_set[["gender","age","hypertension","heart_disease","smoking_status","stroke"]]

In [ ]:
X_train = train_set[["gender","age","hypertension","heart_disease","smoking_status"]]
y_train = train_set["stroke"]

In [ ]:
X_test = test_set[["gender","age","hypertension","heart_disease","smoking_status"]]
y_test = test_set["stroke"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree

In [ ]:
num_attributes = X_train.select_dtypes( include = ['int64','float64']).columns
cat_attributes = X_train.select_dtypes( include = ['object']).columns

In [ ]:
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )

In [ ]:
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier(max_depth=3))])

In [ ]:
cross_val_score( pipeline, X_train, y_train, cv=5, scoring='accuracy')

In [ ]:
model = pipeline.fit( X_train, y_train )
model

In [ ]:
predictions = pipeline.predict( X_test )

In [ ]:
cm = confusion_matrix( y_test, predictions )

In [ ]:
cm

In [ ]:
plot_tree( pipeline.named_steps['clf'])